In [1]:
import cifar10,cifar10_input
import tensorflow as tf
import numpy as np
import time

In [2]:
max_steps = 3000
batch_size = 128
data_dir = '/tmp/cifar10_data/cifar-10-batches-bin'

In [3]:
def variable_with_weight_lose(shape, stddev, w1):
    var = tf.Variable(tf.truncated_normal(shape, stddev=stddev))
    if w1 is not None:
        weight_loss = tf.multiply(tf.nn.l2_loss(var), w1, name='weight_loss')
        tf.add_to_collection('losses', weight_loss)
    return var

In [4]:
cifar10.maybe_download_and_extract()

In [5]:
images_train, labels_train = cifar10_input.distorted_inputs(data_dir=data_dir,batch_size=batch_size)

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.


In [6]:
images_test, lables_test = cifar10_input.inputs(eval_data=True, data_dir=data_dir, batch_size=batch_size)

In [7]:
image_holder = tf.placeholder(tf.float32, [batch_size, 24, 24, 3])
label_holder = tf.placeholder(tf.int32, [batch_size])

In [8]:
#setup first convolution layer
weight1 = variable_with_weight_lose(shape=[5, 5, 3, 64], stddev=5e-2, w1=0.0)
kernel1 = tf.nn.conv2d(image_holder, weight1, [1, 1, 1, 1], padding='SAME')
bias1 = tf.Variable(tf.constant(0.0, shape=[64]))
conv1 = tf.nn.relu(tf.nn.bias_add(kernel1, bias1))
pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)

In [9]:
#setup sencond convolution layer
weight2 = variable_with_weight_lose(shape=[5, 5, 64, 64], stddev=5e-2, w1=0.0)
kernel2 = tf.nn.conv2d(norm1, weight2, [1, 1, 1, 1], padding='SAME')
bias2 = tf.Variable(tf.constant(0.0, shape=[64]))
conv2 = tf.nn.relu(tf.nn.bias_add(kernel2, bias2))
norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

In [10]:
#setup first full connection layer
reshape = tf.reshape(pool2, [batch_size, -1])
dim = reshape.get_shape()[1].value
weight3 = variable_with_weight_lose(shape=[dim, 384], stddev=0.04, w1=0.004)
bias3 = tf.Variable(tf.constant(0.1, shape=[384]))
local3 = tf.nn.relu(tf.matmul(reshape, weight3) + bias3)

In [11]:
#setup sencond full connection layer
weight4 = variable_with_weight_lose(shape=[384, 192], stddev=0.04, w1=0.004)
bias4 = tf.Variable(tf.constant(0.1, shape=[192]))
local4 = tf.nn.relu(tf.matmul(local3, weight4) + bias4)

In [12]:
#setup third full connection layer
weight5 = variable_with_weight_lose(shape=[192, 10], stddev=1/192.0, w1=0.0)
bias5 = tf.Variable(tf.constant(0.0, shape=[10]))
logits = tf.nn.relu(tf.matmul(local4, weight5) + bias5)

In [13]:
#define loss
def loss(logits, labels):
    labels = tf.cast(labels, tf.int64)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels, 
                                                                   name='cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
    tf.add_to_collection('losses', cross_entropy_mean)
    return tf.add_n(tf.get_collection('losses'), name='total_loss')

In [14]:
loss = loss(logits, label_holder)
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)
top_k_op = tf.nn.in_top_k(logits, label_holder, 1)

In [15]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

In [16]:
tf.train.start_queue_runners()

[<Thread(Thread-4, started daemon 140540297332480)>,
 <Thread(Thread-5, started daemon 140540288939776)>,
 <Thread(Thread-6, started daemon 140540280547072)>,
 <Thread(Thread-7, started daemon 140540272154368)>,
 <Thread(Thread-8, started daemon 140540188292864)>,
 <Thread(Thread-9, started daemon 140540179900160)>,
 <Thread(Thread-10, started daemon 140540171507456)>,
 <Thread(Thread-11, started daemon 140540163114752)>,
 <Thread(Thread-12, started daemon 140540154722048)>,
 <Thread(Thread-13, started daemon 140540146329344)>,
 <Thread(Thread-14, started daemon 140540137936640)>,
 <Thread(Thread-15, started daemon 140540054075136)>,
 <Thread(Thread-16, started daemon 140540045682432)>,
 <Thread(Thread-17, started daemon 140540037289728)>,
 <Thread(Thread-18, started daemon 140540028897024)>,
 <Thread(Thread-19, started daemon 140540020504320)>,
 <Thread(Thread-20, started daemon 140540012111616)>,
 <Thread(Thread-21, started daemon 140540003718912)>,
 <Thread(Thread-22, started daemon

ERROR:tensorflow:Exception in QueueRunner: assertion failed: [Need value.shape >= size, got ] [32 32 3] [24 24 3]
	 [[Node: random_crop/Assert/Assert = Assert[T=[DT_STRING, DT_INT32, DT_INT32], summarize=3, _device="/job:localhost/replica:0/task:0/cpu:0"](random_crop/All/_21, random_crop/Assert/Assert/data_0, random_crop/Shape/_23, random_crop/size/_25)]]
	 [[Node: StridedSlice/_39 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_119_StridedSlice", tensor_type=DT_UINT8, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'random_crop/Assert/Assert', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/keyu/.local/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module

In [27]:
for step in range(max_steps):
    start_time = time.time()
    image_batch, label_batch = sess.run([images_train, labels_train])
    _, loss_value = sess.run([train_op, loss],
                             feed_dict={image_holder:image_batch, label_holder:label_batch})
    duration = time.time() - start_time

In [1]:
#if step % 10 == 0:
    examples_per_sec = batch_size /duration
    sec_per_bach = float(duration)
    
    format_str='(step %d, loss %.2f %.1f examples/sec;%.3f sec/batch)
    print(format_str %(step, loss_value, expamples_per_sec, sec_per_batch))

IndentationError: unexpected indent (<ipython-input-1-7270b3bd4ff5>, line 2)

In [ ]:
num_examples = 10000
import math
num_iter = int(math.ceil(num_examples/batch_size))
true_count = 0
total_sample_count = num_iter * batch_size
step = 0
while step < num_iter:
    image_batch, label_batch = sess.run(images_test, labels_test)
    predictions = sess.run([top_k_op], feed_dict={image_holder:image_batch,
                                                 label_holder:label_batch})
    true_count += np.sum(predictions)
    step +=1
precision = true_count /total_sample_count
print('precision @1 = %.3f' %precision)